1. url
2. thumbnil url
3. views
4. likes
5. time posted
6. title
7. save in csv file

In [4]:
api_key = 'Be Ready with yours'


In [5]:
import os
import csv
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

def get_channel_id(api_key, handle):
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    try:
        # Search for the channel using the handle
        search_response = youtube.search().list(
            q=handle,
            part='snippet',
            type='channel',
            maxResults=1
        ).execute()
        
        if not search_response.get('items'):
            print(f"No channel found for handle: {handle}")
            return None
        
        channel_id = search_response['items'][0]['snippet']['channelId']
        return channel_id
    except HttpError as e:
        print(f"An HTTP error occurred: {e}")
        return None

def get_channel_videos(api_key, handle):
    channel_id = get_channel_id(api_key, handle)
    if not channel_id:
        return []

    youtube = build('youtube', 'v3', developerKey=api_key)
    
    # Get the playlist ID of the channel's uploaded videos
    uploads_playlist_id = youtube.channels().list(
        id=channel_id,
        part='contentDetails'
    ).execute()['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    # Retrieve videos from the uploads playlist
    video_details = []
    next_page_token = None

    while True:
        playlist_response = youtube.playlistItems().list(
            playlistId=uploads_playlist_id,
            part='snippet',
            maxResults=50,
            pageToken=next_page_token
        ).execute()
        
        for item in playlist_response['items']:
            video_id = item['snippet']['resourceId']['videoId']
            video_title = item['snippet']['title']
            video_published_at = item['snippet']['publishedAt']
            thumbnail_url = item['snippet']['thumbnails']['high']['url']
            
            # Get video statistics
            video_response = youtube.videos().list(
                id=video_id,
                part='statistics'
            ).execute()
            
            video_views = video_response['items'][0]['statistics'].get('viewCount', 'N/A')
            video_likes = video_response['items'][0]['statistics'].get('likeCount', 'N/A')
            
            video_url = f'https://www.youtube.com/watch?v={video_id}'
            
            video_details.append({
                'URL': video_url,
                'Thumbnail URL': thumbnail_url,
                'Views': video_views,
                'Likes': video_likes,
                'Time Posted': video_published_at,
                'Title': video_title
            })

        next_page_token = playlist_response.get('nextPageToken')
        if not next_page_token:
            break

    return video_details

def save_to_csv(video_details, filename):
    df = pd.DataFrame(video_details)
    df.to_csv(filename, index=False)

def main():
    api_key = input("Enter your YouTube Data API key: ")
    handle = input("Enter the YouTube channel handle (e.g., @PW-Foundation): ")
    
    try:
        video_details = get_channel_videos(api_key, handle)
        if video_details:
            filename = f'{handle[1:]}_videos.csv'  # Remove '@' from the handle for filename
            save_to_csv(video_details, filename)
            print(f"Video details saved to {filename}")
        else:
            print("No videos found or error retrieving videos.")
    except HttpError as e:
        print(f"An HTTP error occurred: {e}")

if __name__ == "__main__":
    main()


Video details saved to PW-Foundation_videos.csv
